#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter ITP Audit Dashboard Parameters
Dashboard that shows performance metrics across browser to see the impact of ITP.
 1. Follow the steps in the below document
 1. https://docs.google.com/document/d/1HaRCMaBBEo0tSKwnofWNtaPjlW0ORcVHVwIRabct4fY/edit?usp=sharing
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'dataset': 'ITP_Audit_Dashboard',  # BigQuery dataset for store dashboard tables.
  'sheet_url': '',  # Sheet URL for the Segments sheet.
  'cm_account_id': '',  # Campaign Manager Account Id.
  'advertiser_ids': '',  # Comma separated list of Campaign Manager Advertiser Ids.  Leave blank for no advertiser filtering.
  'floodlight_configuration_id': '',  # Floodlight Configuration Id for the Campaign Manager floodlight report.
  'floodlight_report_name': 'ITP_Audit_Dashboard_Floodlight',  # Campaign Manager Floodlight report name.
  'dv360_report_name': 'ITP_Audit_Browser_Report',  # DV360 Browser report name.
  'cm_browser_report_name': 'ITP_Audit_Dashboard_Browser',  # Name of the Campaign Manager browser report.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute ITP Audit Dashboard
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'auth': 'user',
      'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}}
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'timeout': 60,
      'report': {
        'account': {'field': {'name': 'cm_account_id','kind': 'string','order': 3,'default': '','description': 'Campaign Manager Account Id.'}},
        'body': {
          'kind': 'dfareporting#report',
          'name': {'field': {'name': 'floodlight_report_name','kind': 'string','order': 8,'default': 'ITP_Audit_Dashboard_Floodlight','description': 'Campaign Manager Floodlight report name.'}},
          'fileName': {'field': {'name': 'floodlight_report_name','kind': 'string','order': 8,'default': 'ITP_Audit_Dashboard_Floodlight','description': 'Campaign Manager Floodlight report name.'}},
          'format': 'CSV',
          'type': 'FLOODLIGHT',
          'floodlightCriteria': {
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_30_DAYS'
            },
            'floodlightConfigId': {
              'kind': 'dfareporting#dimensionValue',
              'dimensionName': 'dfa:floodlightConfigId',
              'value': {'field': {'name': 'floodlight_configuration_id','kind': 'integer','order': 7,'default': '','description': 'Floodlight Configuration Id for the Campaign Manager floodlight report.'}},
              'matchType': 'EXACT'
            },
            'reportProperties': {
              'includeUnattributedIPConversions': False,
              'includeUnattributedCookieConversions': True
            },
            'dimensions': [
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:site'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:floodlightAttributionType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:interactionType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:pathType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:browserPlatform'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:platformType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:week'
              }
            ],
            'metricNames': [
              'dfa:activityClickThroughConversions',
              'dfa:activityViewThroughConversions',
              'dfa:totalConversions',
              'dfa:totalConversionsRevenue'
            ]
          },
          'schedule': {
            'active': True,
            'repeats': 'DAILY',
            'every': 1,
            'startDate': '2019-09-11',
            'expirationDate': '2029-12-10'
          },
          'delivery': {
            'emailOwner': False
          }
        }
      },
      'out': {
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'Floodlight_CM_Report',
          'is_incremental_load': False
        }
      },
      'delete': False
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'datastudio': True,
      'report': {
        'name': {'field': {'name': 'dv360_report_name','kind': 'string','order': 8,'default': 'ITP_Audit_Browser_Report','description': 'DV360 Browser report name.'}}
      },
      'out': {
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'Dv360_Browser_Report_Dirty',
          'autodetect_schema': True,
          'is_incremental_load': False
        }
      }
    }
  },
  {
    'sheets': {
      'auth': 'user',
      'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'default': '','description': 'Sheet URL for the Segments sheet.'}},
      'tab': 'Enviroment',
      'range': 'A:B',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'Environment'
        }
      }
    }
  },
  {
    'sheets': {
      'auth': 'user',
      'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'default': '','description': 'Sheet URL for the Segments sheet.'}},
      'tab': 'Browser',
      'range': 'A:C',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'Browser'
        }
      }
    }
  },
  {
    'sheets': {
      'auth': 'user',
      'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'default': '','description': 'Sheet URL for the Segments sheet.'}},
      'tab': 'CM_Site_Segments',
      'range': 'A:C',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'CM_Browser_lookup'
        }
      }
    }
  },
  {
    'sheets': {
      'auth': 'user',
      'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'default': '','description': 'Sheet URL for the Segments sheet.'}},
      'tab': 'Device_Type',
      'range': 'A:B',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'Device_Type'
        }
      }
    }
  },
  {
    'sheets': {
      'auth': 'user',
      'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'default': '','description': 'Sheet URL for the Segments sheet.'}},
      'tab': 'Floodlight_Attribution',
      'range': 'A:B',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'Floodlight_Attribution'
        }
      }
    }
  },
  {
    'itp_audit': {
      'auth': 'user',
      'account': {'field': {'name': 'cm_account_id','kind': 'string','order': 3,'default': '','description': 'Campaign Manager Account Id.'}},
      'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'default': 'ITP_Audit_Dashboard','description': 'BigQuery dataset for store dashboard tables.'}},
      'sheet': {'field': {'name': 'sheet_url','kind': 'string','order': 2,'default': '','description': 'Sheet URL for the Segments sheet.'}},
      'cm_browser_report_name': {'field': {'name': 'cm_browser_report_name','kind': 'string','order': 9,'default': 'ITP_Audit_Dashboard_Browser','description': 'Name of the Campaign Manager browser report.'}},
      'advertiser_ids': {'field': {'name': 'advertiser_ids','kind': 'string','order': 5,'default': '','description': 'Comma separated list of Campaign Manager Advertiser Ids.  Leave blank for no advertiser filtering.'}},
      'timeout': 60
    }
  }
]

json_set_fields(TASKS, FIELDS)
project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True)
project.execute()
